# Scikit-Learn SVM with NVFLARE


## Prepare data

In this section, we will download the data and split the data and save to the local disk

### Download data

In [1]:
from utils.prepare_data import download_data

The download data function will download one of the two datasets from Scikit-learn: Iris or Cancer
* the file will be save to the output directory 
* the file format will be CSV format with comma separated
* the file will be remove the header 
* default dataset is iris
* filename = dataset name


In [2]:
output_dir="/tmp/nvflare/sklearn/data"
download_data(output_dir)

Verify the file is downloaded


In [3]:
!ls {output_dir}

iris.csv


#### Split Data
* **Split Method**


Split the data into different datasets, one for each client. 
There are several split methods, we use test our algorithms in different scenarios. Here we just pick uniform split from the followns
* Uniform 
* linear
* Sqare
* Exponential



* **data store method**

There are two approaches to store the splited data 
* STORE DATA: 

similar to the real application, we split the data total into different directories (sites), and each client will ready one-site's data

* STORE_INDEX: 

simulate the split, by assign data index range for each site, but the original file is not splited. The data loader is reading from the original file but only for the data within the index range
  For example: the index assignment for the data split is captured in a json file
 ``` 
  {
     "data_path" : "/tmp/nvflare/sklearn/data/iris.csv"
     "data_index" : {
         "site-1": {"start": 100, "end": 300},
         "site-2": {"start": 301, "end": 600},
     }
  }
 ```

Here we choose STORE_DATA approach

In [4]:
from utils.prepare_data_split import split_data, SplitMethod, StoreMethod, get_file_line_count

In [5]:
input_path = f"{output_dir}/iris.csv"
output_dir = "/tmp/nvflare/sklearn/data"
site_num = 3
valid_frac = 0.3
split_method: SplitMethod = SplitMethod.UNIFORM
store_method: StoreMethod = StoreMethod.STORE_DATA

In [9]:

split_data(input_path, output_dir, site_num, valid_frac, split_method=split_method, store_method=store_method)

start type= <class 'float'>
end type= <class 'float'>
start type= <class 'float'>
end type= <class 'float'>
start type= <class 'float'>
end type= <class 'float'>


TypeError: 'float' object cannot be interpreted as an integer

In [20]:
get_file_line_count(input_path)

150